# Objetivo:
#### Este notebook contiene los pasos ejecutados para obtener datos de los precios inmuebles en zonas costeras del Atlántico en EE.UU. y variables economicas y sociodemograficas posiblemente relacionadas.
#### Los datos obtenidos se pueden encontrar en la página oficial del Censo de los Estados Unidos. Esto se lleva a cabo haciendo uso de la [API The American Community Survey (ACS)](https://www.census.gov/data/developers/data-sets/acs-5year.html) que contiene información sobre características de vivienda, económicas y demográficas, entre los años 2009 a 2022.

## 0. Importación de librerías

##### Se debe instalar la librería censusdata para hacer uso de la API

In [2]:
#%pip install censusdata

In [85]:
import requests
import pandas as pd
import ast
import censusdata
import io
import csv

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## 1. Buscador:
##### Se hace un exploración de las variables disponibles

In [86]:
# Analizar que variables hay

buscador = censusdata.search('acs5', 2009,'concept', 'housing units')
for i in buscador:
    print(i)


('B00002_001E', 'UNWEIGHTED SAMPLE HOUSING UNITS', 'Estimate!!Total')
('B25001_001E', 'HOUSING UNITS', 'Estimate!!Total')
('B25008_001E', 'TOTAL POPULATION IN OCCUPIED HOUSING UNITS BY TENURE', 'Estimate!!Total')
('B25008_002E', 'TOTAL POPULATION IN OCCUPIED HOUSING UNITS BY TENURE', 'Estimate!!Total!!Owner occupied')
('B25008_003E', 'TOTAL POPULATION IN OCCUPIED HOUSING UNITS BY TENURE', 'Estimate!!Total!!Renter occupied')
('B25010_001E', 'AVERAGE HOUSEHOLD SIZE OF OCCUPIED HOUSING UNITS BY TENURE', 'Estimate!!Total')
('B25010_002E', 'AVERAGE HOUSEHOLD SIZE OF OCCUPIED HOUSING UNITS BY TENURE', 'Estimate!!Total!!Owner occupied')
('B25010_003E', 'AVERAGE HOUSEHOLD SIZE OF OCCUPIED HOUSING UNITS BY TENURE', 'Estimate!!Total!!Renter occupied')
('B25026_001E', 'TOTAL POPULATION IN OCCUPIED HOUSING UNITS BY TENURE BY YEAR HOUSEHOLDER MOVED INTO UNIT', 'Estimate!!Total population in occupied housing units')
('B25026_002E', 'TOTAL POPULATION IN OCCUPIED HOUSING UNITS BY TENURE BY YEAR HOUSEH

In [87]:
print(buscador[0])
print(buscador[1])

('B00002_001E', 'UNWEIGHTED SAMPLE HOUSING UNITS', 'Estimate!!Total')
('B25001_001E', 'HOUSING UNITS', 'Estimate!!Total')


In [88]:
# Search for ACS 2011-2015 5-year estimate variables where the concept includes the text 'unweighted sample'.
censusdata.search('acs5', 2015, 'concept', 'unweighted sample')
# Search for ACS 2011-2015 5-year estimate variables where the specific variable label includes the text 'unemploy'.
censusdata.search('acs5', 2015, 'label', 'household')

[('B07013PR_002E',
  'GEOGRAPHICAL MOBILITY IN THE PAST YEAR BY TENURE FOR CURRENT RESIDENCE IN PUERTO RICO',
  'Estimate!!Total!!Householder lived in owner-occupied housing units'),
 ('B07013PR_003E',
  'GEOGRAPHICAL MOBILITY IN THE PAST YEAR BY TENURE FOR CURRENT RESIDENCE IN PUERTO RICO',
  'Estimate!!Total!!Householder lived in renter-occupied housing units'),
 ('B07013PR_005E',
  'GEOGRAPHICAL MOBILITY IN THE PAST YEAR BY TENURE FOR CURRENT RESIDENCE IN PUERTO RICO',
  'Estimate!!Total!!Same house 1 year ago!!Householder lived in owner-occupied housing units'),
 ('B07013PR_006E',
  'GEOGRAPHICAL MOBILITY IN THE PAST YEAR BY TENURE FOR CURRENT RESIDENCE IN PUERTO RICO',
  'Estimate!!Total!!Same house 1 year ago!!Householder lived in renter-occupied housing units'),
 ('B07013PR_008E',
  'GEOGRAPHICAL MOBILITY IN THE PAST YEAR BY TENURE FOR CURRENT RESIDENCE IN PUERTO RICO',
  'Estimate!!Total!!Moved within same municipio!!Householder lived in owner-occupied housing units'),
 ('B0701

In [89]:
import re

# Search for ACS 2011-2015 5-year estimate variables where the concept includes the text 'unweighted sample' and the text 'housing'.
censusdata.search('acs5', 2015, 'concept', lambda value: re.search('unweighted sample', value, re.IGNORECASE) and re.search('housing', value, re.IGNORECASE))

[('B00002_001E', 'UNWEIGHTED SAMPLE HOUSING UNITS', 'Estimate!!Total'),
 ('GEO_ID',
  "SEX BY SCHOOL ENROLLMENT BY LEVEL OF SCHOOL BY TYPE OF SCHOOL FOR THE POPULATION 3 YEARS AND OVER;SEX BY SCHOOL ENROLLMENT BY TYPE OF SCHOOL BY AGE FOR THE POPULATION 3 YEARS AND OVER;SEX BY COLLEGE OR GRADUATE SCHOOL ENROLLMENT BY TYPE OF SCHOOL BY AGE FOR THE POPULATION 15 YEARS AND OVER;SEX BY SCHOOL ENROLLMENT BY EDUCATIONAL ATTAINMENT BY EMPLOYMENT STATUS FOR THE POPULATION 16 TO 19 YEARS;POVERTY STATUS IN THE PAST 12 MONTHS BY SCHOOL ENROLLMENT BY LEVEL OF SCHOOL FOR THE POPULATION 3 YEARS AND OVER;SCHOOL ENROLLMENT BY DETAILED  LEVEL OF SCHOOL FOR THE POPULATION 3 YEARS AND OVER;SCHOOL ENROLLMENT BY DETAILED LEVEL OF SCHOOL FOR THE POPULATION 3 YEARS AND OVER (WHITE ALONE);SCHOOL ENROLLMENT BY DETAILED LEVEL OF SCHOOL FOR THE POPULATION 3 YEARS AND OVER (BLACK OR AFRICAN AMERICAN ALONE);SCHOOL ENROLLMENT BY DETAILED LEVEL OF SCHOOL FOR THE POPULATION 3 YEARS AND OVER (AMERICAN INDIAN AND ALASK

In [90]:

counties = censusdata.geographies(censusdata.censusgeo([('state', '12'), ('county', '*')]), 'acs5', 2021)


## 3. DATOS: FLORIDA

In [91]:
# VARIABLES 

# censusdata.printtable(censusdata.censustable('acs5', 2009, 'B25001')) # Total housing units B25001_001E  
# censusdata.printtable(censusdata.censustable('acs5', 2009, 'B25018')) # Median rooms B25018_001E  
# censusdata.printtable(censusdata.censustable('acs5', 2009, 'B25077')) # Median Price (dollars) B25077_001E
# censusdata.printtable(censusdata.censustable('acs5', 2009, 'B01003')) # Total population B01003_001E  
# censusdata.printtable(censusdata.censustable('acs5', 2009, 'B01002')) # Median age (years) B01002_001E
# Falta "Citizen 18 & over population"

# censusdata.printtable(censusdata.censustable('acs5', 2009, 'B25002')) # Vacant housing units B25002_001E    
# censusdata.printtable(censusdata.censustable('acs5', 2009, 'B25002')) # Owner-occupied B25003_002E    
# censusdata.printtable(censusdata.censustable('acs5', 2009, 'B25003')) # Renter-occupied B25003_003E
# Falta "Moved in 2019 or later"
# Falta "With vehicle"

censusdata.printtable(censusdata.censustable('acs5', 2009, 'DP03')) # Population 16 years and over DP03_0001E (NO FUNCIONA)
censusdata.printtable(censusdata.censustable('acs5', 2009, 'DP03')) # Employed DP03_0004EA Population 16 years and over (NO FUNCIONA)
censusdata.printtable(censusdata.censustable('acs5', 2009, 'DP03')) # Unemployed DP03_0005E Population 16 years and over (NO FUNCIONA)
# censusdata.printtable(censusdata.censustable('acs5', 2009, 'B11016')) # Total Households B11016_002E  
# censusdata.printtable(censusdata.censustable('acs5', 2009, 'B19013')) # Median Household income B19013_001E
censusdata.printtable(censusdata.censustable('acs5', 2009, 'DP03')) # Mean Household Income DP03_0063E (NO FUNCIONA)

# censusdata.printtable(censusdata.censustable('acs5', 2009, 'B19113')) # Median Family income B19113_001E
censusdata.printtable(censusdata.censustable('acs5', 2009, 'DP03')) # Mean Family Income DP03_0087E (NO FUNCIONA)
# censusdata.printtable(censusdata.censustable('acs5', 2009, 'B19301')) # Per capita income B19301_001E
# censusdata.printtable(censusdata.censustable('acs5', 2009, 'B11016')) # Nonfamily households B11016_009E
# censusdata.printtable(censusdata.censustable('acs5', 2009, 'B19202')) # Median nonfamily income B19202_001E  
censusdata.printtable(censusdata.censustable('acs5', 2009, 'DP03')) # Mean Nonfamily Income DP03_0091E (NO FUNCIONA)

censusdata.printtable(censusdata.censustable('acs5', 2019, 'DP03')) # Median earnings for workers DP03_0092E (NO FUNCIONA)

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
DP03_0001E   | Selected Economic Characterist | !! !! !! Number Estimate EMPLOYMENT STATUS Population 16 | int  
DP03_0001PE  | Selected Economic Characterist | !! !! !! Percent Estimate EMPLOYMENT STATUS Population 1 | int  
DP03_0002E   | Selected Economic Characterist | !! !! !! !! Number Estimate EMPLOYMENT STATUS Population | int  
DP03_0002PE  | Selected Economic Characterist | !! !! !! !! Percent Estimate EMPLOYMENT STATUS Populatio | float
DP03_0003E   | Selected Economic Characterist | !! !! !! !! !! Number Estimate EMPLOYMENT STATUS Populat | int  
DP03_0003PE  | Selected Economic Characterist | !! !! !! !! !! Percent Estimate EMPLOYMENT STATUS Popula | float
DP03_0004E   | Selected Economic Characterist | !! !! !! !! !! !! Number Estimate EMPLOYMENT 

In [92]:
# Configuración base de la API
host = 'https://api.census.gov/data'
dataset_acronym = '/acs/acs5'
g = '?get='

# Variables que deseas descargar
variables = 'NAME,B25001_001E,B25018_001E,B25077_001E,B01003_001E,B01002_001E,B25002_001E,B25003_002E,B25003_003E,B11016_002E,B19013_001E,B19113_001E,B19301_001E,B11016_009E,B19202_001E,B25064_001E,B19083_001E,B17001_001E,B23025_005E'

# Ubicación (Virginia, FIPS code: 51)
location = '&for=county:*&in=state:12'
usr_key = f"&key=ed3a1808edacc745995c92696b277631b5ec04f9"

# Lista de años
years = list(range(2009, 2024))

# Lista para almacenar los dataframes
dataframes = []

# Iterar sobre cada año y realizar la consulta
for year in years:
    year_str = f"/{year}"
    query_url = f"{host}{year_str}{dataset_acronym}{g}{variables}{location}{usr_key}"
    print(f"Fetching data for year {year}: {query_url}")

    response = requests.get(query_url)
    urlData = response.content

    df = pd.read_csv(io.StringIO(urlData.decode('utf-8')), skiprows=1, names=["County", "State", "Housing_units", "Median_rooms", "Median_Price", "Total_population", "Median_age",
                                                                              "Vacant_housing_units", "Owner_occupied", "Renter_occupied", "Total_Household", "Median_Household_income", "Median_Family_income",
                                                                              "PerCapita_income", "Nonfamily_households", "Median_nonfamily_income", "Median_Gross_Rent","Gini_Index","Poverty_Status","Unemployment_ 16YearsAndOver","# State", "# County", "fuck2"])
    df = df.iloc[:,:-1]
    df['Year'] = year

    # Limpiando las variables
    df['County'] = df['County'].map(lambda x: x.lstrip('["'))
    df['State'] = df['State'].map(lambda x: x.rstrip('"'))
    df['# County'] = df['# County'].map(lambda x: x.rstrip(']'))
    
    # Añadir el dataframe a la lista
    dataframes.append(df)

# Concatenar todos los dataframes en uno solo
final_df = pd.concat(dataframes, ignore_index=True)
final_df.head()

Fetching data for year 2009: https://api.census.gov/data/2009/acs/acs5?get=NAME,B25001_001E,B25018_001E,B25077_001E,B01003_001E,B01002_001E,B25002_001E,B25003_002E,B25003_003E,B11016_002E,B19013_001E,B19113_001E,B19301_001E,B11016_009E,B19202_001E,B25064_001E,B19083_001E,B17001_001E,B23025_005E&for=county:*&in=state:12&key=ed3a1808edacc745995c92696b277631b5ec04f9
Fetching data for year 2010: https://api.census.gov/data/2010/acs/acs5?get=NAME,B25001_001E,B25018_001E,B25077_001E,B01003_001E,B01002_001E,B25002_001E,B25003_002E,B25003_003E,B11016_002E,B19013_001E,B19113_001E,B19301_001E,B11016_009E,B19202_001E,B25064_001E,B19083_001E,B17001_001E,B23025_005E&for=county:*&in=state:12&key=ed3a1808edacc745995c92696b277631b5ec04f9
Fetching data for year 2011: https://api.census.gov/data/2011/acs/acs5?get=NAME,B25001_001E,B25018_001E,B25077_001E,B01003_001E,B01002_001E,B25002_001E,B25003_002E,B25003_003E,B11016_002E,B19013_001E,B19113_001E,B19301_001E,B11016_009E,B19202_001E,B25064_001E,B19083_0

/var/folders/rv/dlx920mn2hj15wkt3g1nkdkn3dpgn4/T/ipykernel_4536/628119603.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(dataframes, ignore_index=True)


,County,State,Housing_units,Median_rooms,Median_Price,Total_population,Median_age,Vacant_housing_units,Owner_occupied,Renter_occupied,Total_Household,Median_Household_income,Median_Family_income,PerCapita_income,Nonfamily_households,Median_nonfamily_income,Median_Gross_Rent,Gini_Index,Poverty_Status,Unemployment_ 16YearsAndOver,# State,# County,Year
0,Martin County,Florida,77898,5.3,238200,145480,49.3,77898,46746,12570,37597,53612,71484,35822,21719,31785,992,0.5102,142450,7173,12,085,2011
1,Sarasota County,Florida,227606,5.2,213400,378404,52.1,227606,128934,40322,103990,49212,62148,33096,65266,32221,1013,0.4830,372765,18202,12,115,2011
2,Citrus County,Florida,77587,5.2,134800,141157,53.8,77587,49936,9979,39352,38189,45921,22939,20563,23876,770,0.4390,138733,7956,12,017,2011
3,Santa Rosa County,Florida,64066,5.8,173400,150073,39.0,64066,42695,13261,41824,55913,63796,26507,14132,32870,956,0.4075,146588,7155,12,113,2011
4,Broward County,Florida,809226,4.7,225300,1742012,39.4,809226,453419,211618,417862,51782,62852,28720,247175,34312,1162,0.4720,1725510,99062,12,011,2011


In [93]:
final_df.shape

(804, 23)

In [94]:
final_df.groupby('Year').count()

,County,State,Housing_units,Median_rooms,Median_Price,Total_population,Median_age,Vacant_housing_units,Owner_occupied,Renter_occupied,Total_Household,Median_Household_income,Median_Family_income,PerCapita_income,Nonfamily_households,Median_nonfamily_income,Median_Gross_Rent,Gini_Index,Poverty_Status,Unemployment_ 16YearsAndOver,# State,# County
Year,,,,,,,,,,,,,,,,,,,,,,
2011,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67
2012,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67
2013,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67
2014,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67
2015,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67
2016,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67
2017,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67
2018,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67
2019,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67


In [95]:
final_df.County.nunique()

67

In [96]:
final_df.to_excel("../data/raw_data/Florida_Data_By_County.xlsx")

## 4. DATOS: VIRGINIA

In [97]:
# Configuración base de la API
host = 'https://api.census.gov/data'
dataset_acronym = '/acs/acs5'
g = '?get='

# Variables que deseas descargar
variables = 'NAME,B25001_001E,B25018_001E,B25077_001E,B01003_001E,B01002_001E,B25002_001E,B25003_002E,B25003_003E,B11016_002E,B19013_001E,B19113_001E,B19301_001E,B11016_009E,B19202_001E,B25064_001E,B19083_001E,B17001_001E,B23025_005E'

# Ubicación (Virginia, FIPS code: 51)
location = '&for=county:*&in=state:51'
usr_key = f"&key=ed3a1808edacc745995c92696b277631b5ec04f9"

# Lista de años
years = list(range(2009, 2024))

# Lista para almacenar los dataframes
dataframes = []

# Iterar sobre cada año y realizar la consulta
for year in years:
    year_str = f"/{year}"
    query_url = f"{host}{year_str}{dataset_acronym}{g}{variables}{location}{usr_key}"
    print(f"Fetching data for year {year}: {query_url}")

    response = requests.get(query_url)
    urlData = response.content

    df = pd.read_csv(io.StringIO(urlData.decode('utf-8')), skiprows=1, names=["County", "State", "Housing_units", "Median_rooms", "Median_Price", "Total_population", "Median_age",
                                                                              "Vacant_housing_units", "Owner_occupied", "Renter_occupied", "Total_Household", "Median_Household_income", "Median_Family_income",
                                                                              "PerCapita_income", "Nonfamily_households", "Median_nonfamily_income", "Median_Gross_Rent","Gini_Index","Poverty_Status","Unemployment_ 16YearsAndOver","# State", "# County", "fuck2"])
    df = df.iloc[:,:-1]
    df['Year'] = year

    # Limpiando las variables
    df['County'] = df['County'].map(lambda x: x.lstrip('["'))
    df['State'] = df['State'].map(lambda x: x.rstrip('"'))
    df['# County'] = df['# County'].map(lambda x: x.rstrip(']'))
    
    # Añadir el dataframe a la lista
    dataframes.append(df)

# Concatenar todos los dataframes en uno solo
final_df = pd.concat(dataframes, ignore_index=True)
final_df.head()

Fetching data for year 2009: https://api.census.gov/data/2009/acs/acs5?get=NAME,B25001_001E,B25018_001E,B25077_001E,B01003_001E,B01002_001E,B25002_001E,B25003_002E,B25003_003E,B11016_002E,B19013_001E,B19113_001E,B19301_001E,B11016_009E,B19202_001E,B25064_001E,B19083_001E,B17001_001E,B23025_005E&for=county:*&in=state:51&key=ed3a1808edacc745995c92696b277631b5ec04f9
Fetching data for year 2010: https://api.census.gov/data/2010/acs/acs5?get=NAME,B25001_001E,B25018_001E,B25077_001E,B01003_001E,B01002_001E,B25002_001E,B25003_002E,B25003_003E,B11016_002E,B19013_001E,B19113_001E,B19301_001E,B11016_009E,B19202_001E,B25064_001E,B19083_001E,B17001_001E,B23025_005E&for=county:*&in=state:51&key=ed3a1808edacc745995c92696b277631b5ec04f9
Fetching data for year 2011: https://api.census.gov/data/2011/acs/acs5?get=NAME,B25001_001E,B25018_001E,B25077_001E,B01003_001E,B01002_001E,B25002_001E,B25003_002E,B25003_003E,B11016_002E,B19013_001E,B19113_001E,B19301_001E,B11016_009E,B19202_001E,B25064_001E,B19083_0

/var/folders/rv/dlx920mn2hj15wkt3g1nkdkn3dpgn4/T/ipykernel_4536/4066520601.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(dataframes, ignore_index=True)


,County,State,Housing_units,Median_rooms,Median_Price,Total_population,Median_age,Vacant_housing_units,Owner_occupied,Renter_occupied,Total_Household,Median_Household_income,Median_Family_income,PerCapita_income,Nonfamily_households,Median_nonfamily_income,Median_Gross_Rent,Gini_Index,Poverty_Status,Unemployment_ 16YearsAndOver,# State,# County,Year
0,James City County,Virginia,29434,6.6,347600,65818,44.6,29434,20000,6356,19199,75938,89537,39311,7157,49583,1128,0.4149,65012,1480,51,095,2011
1,Russell County,Virginia,13499,5.4,89000,28870,42.6,13499,8666,2636,7920,32555,42339,18331,3382,15137,503,0.4615,28552,902,51,167,2011
2,Smyth County,Virginia,15438,5.3,87400,32268,43.1,15438,9055,3626,8557,34533,43194,20626,4124,18811,514,0.4696,31568,979,51,173,2011
3,Bedford city,Virginia,3013,5.3,148000,6184,44.1,3013,1735,1041,1770,34647,43956,21533,1006,22193,707,0.4512,5925,206,51,515,2011
4,Lexington city,Virginia,1945,5.9,253700,6998,21.6,1945,880,847,833,33670,60033,15088,894,23807,695,0.5108,3566,35,51,678,2011


In [98]:
final_df.shape

(1599, 23)

In [99]:
final_df.groupby('Year').count()

,County,State,Housing_units,Median_rooms,Median_Price,Total_population,Median_age,Vacant_housing_units,Owner_occupied,Renter_occupied,Total_Household,Median_Household_income,Median_Family_income,PerCapita_income,Nonfamily_households,Median_nonfamily_income,Median_Gross_Rent,Gini_Index,Poverty_Status,Unemployment_ 16YearsAndOver,# State,# County
Year,,,,,,,,,,,,,,,,,,,,,,
2011,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134
2012,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134
2013,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134
2014,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133
2015,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133
2016,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133
2017,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133
2018,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133
2019,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133,133


In [100]:
final_df.to_excel("../data/raw_data/Virginia_Data_By_County.xlsx")

## 5. DATOS: NEW YORK

In [101]:
# Configuración base de la API
host = 'https://api.census.gov/data'
dataset_acronym = '/acs/acs5'
g = '?get='

# Variables que deseas descargar
variables = 'NAME,B25001_001E,B25018_001E,B25077_001E,B01003_001E,B01002_001E,B25002_001E,B25003_002E,B25003_003E,B11016_002E,B19013_001E,B19113_001E,B19301_001E,B11016_009E,B19202_001E,B25064_001E,B19083_001E,B17001_001E,B23025_005E'

# Ubicación (Virginia, FIPS code: 51)
location = '&for=county:*&in=state:36'
usr_key = f"&key=ed3a1808edacc745995c92696b277631b5ec04f9"

# Lista de años
years = list(range(2009, 2024))

# Lista para almacenar los dataframes
dataframes = []

# Iterar sobre cada año y realizar la consulta
for year in years:
    year_str = f"/{year}"
    query_url = f"{host}{year_str}{dataset_acronym}{g}{variables}{location}{usr_key}"
    print(f"Fetching data for year {year}: {query_url}")

    response = requests.get(query_url)
    urlData = response.content

    df = pd.read_csv(io.StringIO(urlData.decode('utf-8')), skiprows=1, names=["County", "State", "Housing_units", "Median_rooms", "Median_Price", "Total_population", "Median_age",
                                                                              "Vacant_housing_units", "Owner_occupied", "Renter_occupied", "Total_Household", "Median_Household_income", "Median_Family_income",
                                                                              "PerCapita_income", "Nonfamily_households", "Median_nonfamily_income", "Median_Gross_Rent","Gini_Index","Poverty_Status","Unemployment_ 16YearsAndOver","# State", "# County", "fuck2"])
    df = df.iloc[:,:-1]
    df['Year'] = year

    # Limpiando las variables
    df['County'] = df['County'].map(lambda x: x.lstrip('["'))
    df['State'] = df['State'].map(lambda x: x.rstrip('"'))
    df['# County'] = df['# County'].map(lambda x: x.rstrip(']'))
    
    # Añadir el dataframe a la lista
    dataframes.append(df)

# Concatenar todos los dataframes en uno solo
final_df = pd.concat(dataframes, ignore_index=True)
final_df.head()

Fetching data for year 2009: https://api.census.gov/data/2009/acs/acs5?get=NAME,B25001_001E,B25018_001E,B25077_001E,B01003_001E,B01002_001E,B25002_001E,B25003_002E,B25003_003E,B11016_002E,B19013_001E,B19113_001E,B19301_001E,B11016_009E,B19202_001E,B25064_001E,B19083_001E,B17001_001E,B23025_005E&for=county:*&in=state:36&key=ed3a1808edacc745995c92696b277631b5ec04f9
Fetching data for year 2010: https://api.census.gov/data/2010/acs/acs5?get=NAME,B25001_001E,B25018_001E,B25077_001E,B01003_001E,B01002_001E,B25002_001E,B25003_002E,B25003_003E,B11016_002E,B19013_001E,B19113_001E,B19301_001E,B11016_009E,B19202_001E,B25064_001E,B19083_001E,B17001_001E,B23025_005E&for=county:*&in=state:36&key=ed3a1808edacc745995c92696b277631b5ec04f9
Fetching data for year 2011: https://api.census.gov/data/2011/acs/acs5?get=NAME,B25001_001E,B25018_001E,B25077_001E,B01003_001E,B01002_001E,B25002_001E,B25003_002E,B25003_003E,B11016_002E,B19013_001E,B19113_001E,B19301_001E,B11016_009E,B19202_001E,B25064_001E,B19083_0

/var/folders/rv/dlx920mn2hj15wkt3g1nkdkn3dpgn4/T/ipykernel_4536/3419667520.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(dataframes, ignore_index=True)


,County,State,Housing_units,Median_rooms,Median_Price,Total_population,Median_age,Vacant_housing_units,Owner_occupied,Renter_occupied,Total_Household,Median_Household_income,Median_Family_income,PerCapita_income,Nonfamily_households,Median_nonfamily_income,Median_Gross_Rent,Gini_Index,Poverty_Status,Unemployment_ 16YearsAndOver,# State,# County,Year
0,Bronx County,New York,511180,3.9,391300,1374593,32.6,511180,94903,377020,310909,34744,39120,17992,161014,22950,966,0.4842,1338297,81000,36,005,2011
1,Onondaga County,New York,202150,6.0,128600,464921,38.5,202150,120744,62637,113954,52636,68375,27960,69427,30709,734,0.4508,448606,15987,36,067,2011
2,Clinton County,New York,35705,5.5,119600,82265,38.6,35705,21839,9688,19987,49260,62250,23446,11540,26950,693,0.4206,74092,3150,36,019,2011
3,Seneca County,New York,15955,5.9,90600,35306,41.0,15955,9988,3269,8919,47266,55453,21980,4338,27087,664,0.3933,32269,969,36,099,2011
4,Putnam County,New York,38073,6.2,406000,99662,41.2,38073,29202,5796,26597,92711,105391,39746,8401,45023,1268,0.4087,98581,3404,36,079,2011


In [102]:
final_df.shape

(744, 23)

In [103]:
final_df.groupby('Year').count()

,County,State,Housing_units,Median_rooms,Median_Price,Total_population,Median_age,Vacant_housing_units,Owner_occupied,Renter_occupied,Total_Household,Median_Household_income,Median_Family_income,PerCapita_income,Nonfamily_households,Median_nonfamily_income,Median_Gross_Rent,Gini_Index,Poverty_Status,Unemployment_ 16YearsAndOver,# State,# County
Year,,,,,,,,,,,,,,,,,,,,,,
2011,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62
2012,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62
2013,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62
2014,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62
2015,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62
2016,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62
2017,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62
2018,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62
2019,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62


In [104]:
final_df.to_excel("../data/raw_data/NewYork_Data_By_County.xlsx")

## 6. DATOS: NEW JERSEY

In [105]:
# Configuración base de la API
host = 'https://api.census.gov/data'
dataset_acronym = '/acs/acs5'
g = '?get='

# Variables que deseas descargar
variables = 'NAME,B25001_001E,B25018_001E,B25077_001E,B01003_001E,B01002_001E,B25002_001E,B25003_002E,B25003_003E,B11016_002E,B19013_001E,B19113_001E,B19301_001E,B11016_009E,B19202_001E,B25064_001E,B19083_001E,B17001_001E,B23025_005E'

# Ubicación (Virginia, FIPS code: 51)
location = '&for=county:*&in=state:34'
usr_key = f"&key=ed3a1808edacc745995c92696b277631b5ec04f9"

# Lista de años
years = list(range(2009, 2024))

# Lista para almacenar los dataframes
dataframes = []

# Iterar sobre cada año y realizar la consulta
for year in years:
    year_str = f"/{year}"
    query_url = f"{host}{year_str}{dataset_acronym}{g}{variables}{location}{usr_key}"
    print(f"Fetching data for year {year}: {query_url}")

    response = requests.get(query_url)
    urlData = response.content

    df = pd.read_csv(io.StringIO(urlData.decode('utf-8')), skiprows=1, names=["County", "State", "Housing_units", "Median_rooms", "Median_Price", "Total_population", "Median_age",
                                                                              "Vacant_housing_units", "Owner_occupied", "Renter_occupied", "Total_Household", "Median_Household_income", "Median_Family_income",
                                                                              "PerCapita_income", "Nonfamily_households", "Median_nonfamily_income", "Median_Gross_Rent","Gini_Index","Poverty_Status","Unemployment_ 16YearsAndOver","# State", "# County", "fuck2"])
    df = df.iloc[:,:-1]
    df['Year'] = year

    # Limpiando las variables
    df['County'] = df['County'].map(lambda x: x.lstrip('["'))
    df['State'] = df['State'].map(lambda x: x.rstrip('"'))
    df['# County'] = df['# County'].map(lambda x: x.rstrip(']'))
    
    # Añadir el dataframe a la lista
    dataframes.append(df)

# Concatenar todos los dataframes en uno solo
final_df = pd.concat(dataframes, ignore_index=True)
final_df.head()

Fetching data for year 2009: https://api.census.gov/data/2009/acs/acs5?get=NAME,B25001_001E,B25018_001E,B25077_001E,B01003_001E,B01002_001E,B25002_001E,B25003_002E,B25003_003E,B11016_002E,B19013_001E,B19113_001E,B19301_001E,B11016_009E,B19202_001E,B25064_001E,B19083_001E,B17001_001E,B23025_005E&for=county:*&in=state:34&key=ed3a1808edacc745995c92696b277631b5ec04f9
Fetching data for year 2010: https://api.census.gov/data/2010/acs/acs5?get=NAME,B25001_001E,B25018_001E,B25077_001E,B01003_001E,B01002_001E,B25002_001E,B25003_002E,B25003_003E,B11016_002E,B19013_001E,B19113_001E,B19301_001E,B11016_009E,B19202_001E,B25064_001E,B19083_001E,B17001_001E,B23025_005E&for=county:*&in=state:34&key=ed3a1808edacc745995c92696b277631b5ec04f9
Fetching data for year 2011: https://api.census.gov/data/2011/acs/acs5?get=NAME,B25001_001E,B25018_001E,B25077_001E,B01003_001E,B01002_001E,B25002_001E,B25003_002E,B25003_003E,B11016_002E,B19013_001E,B19113_001E,B19301_001E,B11016_009E,B19202_001E,B25064_001E,B19083_0

/var/folders/rv/dlx920mn2hj15wkt3g1nkdkn3dpgn4/T/ipykernel_4536/1662745432.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(dataframes, ignore_index=True)


,County,State,Housing_units,Median_rooms,Median_Price,Total_population,Median_age,Vacant_housing_units,Owner_occupied,Renter_occupied,Total_Household,Median_Household_income,Median_Family_income,PerCapita_income,Nonfamily_households,Median_nonfamily_income,Median_Gross_Rent,Gini_Index,Poverty_Status,Unemployment_ 16YearsAndOver,# State,# County,Year
0,Mercer County,New Jersey,142808,5.9,302100,365318,37.7,142808,87752,42181,87734,73883,92817,36721,42199,40748,1083,0.4737,345534,17300,34,021,2011
1,Cumberland County,New Jersey,55659,5.6,177800,156142,36.7,55659,34808,16060,35333,52004,61801,22636,15535,29916,914,0.4320,145319,9390,34,011,2011
2,Somerset County,New Jersey,122838,6.5,420500,321304,39.9,122838,90645,23703,83131,98842,117650,48090,31217,55182,1379,0.4491,317929,11119,34,035,2011
3,Ocean County,New Jersey,277087,5.8,284100,572991,42.5,277087,182548,40315,151664,60712,75276,30257,71199,32231,1293,0.4279,565080,24534,34,029,2011
4,Atlantic County,New Jersey,126345,5.6,256600,273674,39.7,126345,71670,29748,67468,55222,68652,27613,33950,32153,995,0.4468,265698,15560,34,001,2011


In [106]:
final_df.shape

(252, 23)

In [107]:
final_df.groupby('Year').count()

,County,State,Housing_units,Median_rooms,Median_Price,Total_population,Median_age,Vacant_housing_units,Owner_occupied,Renter_occupied,Total_Household,Median_Household_income,Median_Family_income,PerCapita_income,Nonfamily_households,Median_nonfamily_income,Median_Gross_Rent,Gini_Index,Poverty_Status,Unemployment_ 16YearsAndOver,# State,# County
Year,,,,,,,,,,,,,,,,,,,,,,
2011,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21
2012,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21
2013,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21
2014,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21
2015,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21
2016,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21
2017,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21
2018,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21
2019,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21


In [108]:
final_df.to_excel("../data/raw_data/NewJersey_Data_By_County.xlsx")

## 7. DATOS: SOUTH CAROLINA

In [109]:
# Configuración base de la API
host = 'https://api.census.gov/data'
dataset_acronym = '/acs/acs5'
g = '?get='

# Variables que deseas descargar
variables = 'NAME,B25001_001E,B25018_001E,B25077_001E,B01003_001E,B01002_001E,B25002_001E,B25003_002E,B25003_003E,B11016_002E,B19013_001E,B19113_001E,B19301_001E,B11016_009E,B19202_001E,B25064_001E,B19083_001E,B17001_001E,B23025_005E'

# Ubicación (Virginia, FIPS code: 51)
location = '&for=county:*&in=state:45'
usr_key = f"&key=ed3a1808edacc745995c92696b277631b5ec04f9"

# Lista de años
years = list(range(2009, 2024))

# Lista para almacenar los dataframes
dataframes = []

# Iterar sobre cada año y realizar la consulta
for year in years:
    year_str = f"/{year}"
    query_url = f"{host}{year_str}{dataset_acronym}{g}{variables}{location}{usr_key}"
    print(f"Fetching data for year {year}: {query_url}")

    response = requests.get(query_url)
    urlData = response.content

    df = pd.read_csv(io.StringIO(urlData.decode('utf-8')), skiprows=1, names=["County", "State", "Housing_units", "Median_rooms", "Median_Price", "Total_population", "Median_age",
                                                                              "Vacant_housing_units", "Owner_occupied", "Renter_occupied", "Total_Household", "Median_Household_income", "Median_Family_income",
                                                                              "PerCapita_income", "Nonfamily_households", "Median_nonfamily_income", "Median_Gross_Rent","Gini_Index","Poverty_Status","Unemployment_ 16YearsAndOver","# State", "# County", "fuck2"])
    df = df.iloc[:,:-1]
    df['Year'] = year

    # Limpiando las variables
    df['County'] = df['County'].map(lambda x: x.lstrip('["'))
    df['State'] = df['State'].map(lambda x: x.rstrip('"'))
    df['# County'] = df['# County'].map(lambda x: x.rstrip(']'))
    
    # Añadir el dataframe a la lista
    dataframes.append(df)

# Concatenar todos los dataframes en uno solo
final_df = pd.concat(dataframes, ignore_index=True)
final_df.head()

Fetching data for year 2009: https://api.census.gov/data/2009/acs/acs5?get=NAME,B25001_001E,B25018_001E,B25077_001E,B01003_001E,B01002_001E,B25002_001E,B25003_002E,B25003_003E,B11016_002E,B19013_001E,B19113_001E,B19301_001E,B11016_009E,B19202_001E,B25064_001E,B19083_001E,B17001_001E,B23025_005E&for=county:*&in=state:45&key=ed3a1808edacc745995c92696b277631b5ec04f9
Fetching data for year 2010: https://api.census.gov/data/2010/acs/acs5?get=NAME,B25001_001E,B25018_001E,B25077_001E,B01003_001E,B01002_001E,B25002_001E,B25003_002E,B25003_003E,B11016_002E,B19013_001E,B19113_001E,B19301_001E,B11016_009E,B19202_001E,B25064_001E,B19083_001E,B17001_001E,B23025_005E&for=county:*&in=state:45&key=ed3a1808edacc745995c92696b277631b5ec04f9
Fetching data for year 2011: https://api.census.gov/data/2011/acs/acs5?get=NAME,B25001_001E,B25018_001E,B25077_001E,B01003_001E,B01002_001E,B25002_001E,B25003_002E,B25003_003E,B11016_002E,B19013_001E,B19113_001E,B19301_001E,B11016_009E,B19202_001E,B25064_001E,B19083_0

/var/folders/rv/dlx920mn2hj15wkt3g1nkdkn3dpgn4/T/ipykernel_4536/2266978878.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(dataframes, ignore_index=True)


,County,State,Housing_units,Median_rooms,Median_Price,Total_population,Median_age,Vacant_housing_units,Owner_occupied,Renter_occupied,Total_Household,Median_Household_income,Median_Family_income,PerCapita_income,Nonfamily_households,Median_nonfamily_income,Median_Gross_Rent,Gini_Index,Poverty_Status,Unemployment_ 16YearsAndOver,# State,# County,Year
0,Aiken County,South Carolina,71685,5.7,122300,158253,40.0,71685,46191,17223,43521,43999,56403,24648,19893,25768,657,0.4676,155663,6137,45,003,2011
1,Allendale County,South Carolina,4506,5.4,72400,10548,37.8,4506,2052,1296,2168,22982,26120,13684,1180,12669,553,0.5687,9280,881,45,005,2011
2,Edgefield County,South Carolina,10487,5.9,104400,26782,40.2,10487,7043,2108,6671,44090,58812,20549,2480,19129,550,0.4461,23988,999,45,037,2011
3,Chester County,South Carolina,14727,5.4,84300,33190,40.4,14727,9560,3045,8639,32112,41056,17929,3966,18265,578,0.4677,32660,2364,45,023,2011
4,Colleton County,South Carolina,19810,5.5,92600,38833,40.0,19810,11136,4145,10253,31700,41221,17702,5028,18700,646,0.4567,38395,2610,45,029,2011


In [110]:
final_df.shape

(552, 23)

In [111]:
final_df.groupby('Year').count()

,County,State,Housing_units,Median_rooms,Median_Price,Total_population,Median_age,Vacant_housing_units,Owner_occupied,Renter_occupied,Total_Household,Median_Household_income,Median_Family_income,PerCapita_income,Nonfamily_households,Median_nonfamily_income,Median_Gross_Rent,Gini_Index,Poverty_Status,Unemployment_ 16YearsAndOver,# State,# County
Year,,,,,,,,,,,,,,,,,,,,,,
2011,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46
2012,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46
2013,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46
2014,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46
2015,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46
2016,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46
2017,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46
2018,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46
2019,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46


In [112]:
final_df.to_excel("../data/raw_data/SouthCarolina_Data_By_County.xlsx")

## 8. DATOS: NORTH CAROLINA

In [113]:
# Configuración base de la API
host = 'https://api.census.gov/data'
dataset_acronym = '/acs/acs5'
g = '?get='

# Variables que deseas descargar
variables = 'NAME,B25001_001E,B25018_001E,B25077_001E,B01003_001E,B01002_001E,B25002_001E,B25003_002E,B25003_003E,B11016_002E,B19013_001E,B19113_001E,B19301_001E,B11016_009E,B19202_001E,B25064_001E,B19083_001E,B17001_001E,B23025_005E'

# Ubicación (Virginia, FIPS code: 51)
location = '&for=county:*&in=state:37'
usr_key = f"&key=ed3a1808edacc745995c92696b277631b5ec04f9"

# Lista de años
years = list(range(2009, 2024))

# Lista para almacenar los dataframes
dataframes = []

# Iterar sobre cada año y realizar la consulta
for year in years:
    year_str = f"/{year}"
    query_url = f"{host}{year_str}{dataset_acronym}{g}{variables}{location}{usr_key}"
    print(f"Fetching data for year {year}: {query_url}")

    response = requests.get(query_url)
    urlData = response.content

    df = pd.read_csv(io.StringIO(urlData.decode('utf-8')), skiprows=1, names=["County", "State", "Housing_units", "Median_rooms", "Median_Price", "Total_population", "Median_age",
                                                                              "Vacant_housing_units", "Owner_occupied", "Renter_occupied", "Total_Household", "Median_Household_income", "Median_Family_income",
                                                                              "PerCapita_income", "Nonfamily_households", "Median_nonfamily_income", "Median_Gross_Rent","Gini_Index","Poverty_Status","Unemployment_ 16YearsAndOver","# State", "# County", "fuck2"])
    df = df.iloc[:,:-1]
    df['Year'] = year

    # Limpiando las variables
    df['County'] = df['County'].map(lambda x: x.lstrip('["'))
    df['State'] = df['State'].map(lambda x: x.rstrip('"'))
    df['# County'] = df['# County'].map(lambda x: x.rstrip(']'))
    
    # Añadir el dataframe a la lista
    dataframes.append(df)

# Concatenar todos los dataframes en uno solo
final_df = pd.concat(dataframes, ignore_index=True)
final_df.head()

Fetching data for year 2009: https://api.census.gov/data/2009/acs/acs5?get=NAME,B25001_001E,B25018_001E,B25077_001E,B01003_001E,B01002_001E,B25002_001E,B25003_002E,B25003_003E,B11016_002E,B19013_001E,B19113_001E,B19301_001E,B11016_009E,B19202_001E,B25064_001E,B19083_001E,B17001_001E,B23025_005E&for=county:*&in=state:37&key=ed3a1808edacc745995c92696b277631b5ec04f9
Fetching data for year 2010: https://api.census.gov/data/2010/acs/acs5?get=NAME,B25001_001E,B25018_001E,B25077_001E,B01003_001E,B01002_001E,B25002_001E,B25003_002E,B25003_003E,B11016_002E,B19013_001E,B19113_001E,B19301_001E,B11016_009E,B19202_001E,B25064_001E,B19083_001E,B17001_001E,B23025_005E&for=county:*&in=state:37&key=ed3a1808edacc745995c92696b277631b5ec04f9
Fetching data for year 2011: https://api.census.gov/data/2011/acs/acs5?get=NAME,B25001_001E,B25018_001E,B25077_001E,B01003_001E,B01002_001E,B25002_001E,B25003_002E,B25003_003E,B11016_002E,B19013_001E,B19113_001E,B19301_001E,B11016_009E,B19202_001E,B25064_001E,B19083_0

/var/folders/rv/dlx920mn2hj15wkt3g1nkdkn3dpgn4/T/ipykernel_4536/1819033289.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(dataframes, ignore_index=True)


,County,State,Housing_units,Median_rooms,Median_Price,Total_population,Median_age,Vacant_housing_units,Owner_occupied,Renter_occupied,Total_Household,Median_Household_income,Median_Family_income,PerCapita_income,Nonfamily_households,Median_nonfamily_income,Median_Gross_Rent,Gini_Index,Poverty_Status,Unemployment_ 16YearsAndOver,# State,# County,Year
0,Clay County,North Carolina,7080,5.4,162100,10506,49.2,7080,3727,737,3020,36711,43067,20758,1444,19175.0,584,0.4553,10380,532,37,043,2011
1,Cumberland County,North Carolina,134705,5.5,123400,316478,30.9,134705,68313,49804,81028,44861,51760,22888,37089,30753.0,820,0.4354,302057,15705,37,051,2011
2,Guilford County,North Carolina,216137,5.5,156200,483081,36.3,216137,120778,71286,119849,46288,59962,26644,72215,29520.0,729,0.4760,469463,25793,37,081,2011
3,Jackson County,North Carolina,25631,5.1,165800,39574,36.4,25631,10500,5259,9378,36826,49761,20226,6381,21260.0,618,0.4551,35995,1353,37,099,2011
4,Pasquotank County,North Carolina,16691,5.7,174000,40511,36.0,16691,9544,5006,10247,45298,56995,23573,4303,24976.0,763,0.4889,38228,2473,37,139,2011


In [114]:
final_df.shape

(1200, 23)

In [115]:
final_df.groupby('Year').count()

,County,State,Housing_units,Median_rooms,Median_Price,Total_population,Median_age,Vacant_housing_units,Owner_occupied,Renter_occupied,Total_Household,Median_Household_income,Median_Family_income,PerCapita_income,Nonfamily_households,Median_nonfamily_income,Median_Gross_Rent,Gini_Index,Poverty_Status,Unemployment_ 16YearsAndOver,# State,# County
Year,,,,,,,,,,,,,,,,,,,,,,
2011,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
2012,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
2013,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
2014,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
2015,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,99,100,100,100,100,100,100
2016,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
2017,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
2018,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
2019,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100


In [116]:
final_df.to_excel("../data/raw_data/NorthCarolina_Data_By_County.xlsx")

## 9. JOIN DATA

In [117]:
data = pd.read_excel("../data/raw_data/Florida_Data_By_County.xlsx")
data2 = pd.read_excel("../data/raw_data/NewJersey_Data_By_County.xlsx")
data3 = pd.read_excel("../data/raw_data/NewYork_Data_By_County.xlsx")
data4 = pd.read_excel("../data/raw_data/NorthCarolina_Data_By_County.xlsx")
data5 = pd.read_excel("../data/raw_data/SouthCarolina_Data_By_County.xlsx")
data6 = pd.read_excel("../data/raw_data/Virginia_Data_By_County.xlsx")

In [118]:
# Sort by two columns

finaldf = pd.concat([data, data2, data3, data4, data5, data6,])

finaldf = finaldf.sort_values(['State', 'Year'], ascending = [True, True])
finaldf = finaldf.drop('Unnamed: 0',axis=1)
finaldf.head()

# finaldf.set_index(['Year', "County"], inplace=True)
# finaldf

,County,State,Housing_units,Median_rooms,Median_Price,Total_population,Median_age,Vacant_housing_units,Owner_occupied,Renter_occupied,Total_Household,Median_Household_income,Median_Family_income,PerCapita_income,Nonfamily_households,Median_nonfamily_income,Median_Gross_Rent,Gini_Index,Poverty_Status,Unemployment_ 16YearsAndOver,# State,# County,Year
0,Martin County,Florida,77898,5.3,238200,145480,49.3,77898,46746,12570,37597,53612,71484,35822,21719,31785.0,992,0.5102,142450,7173,12,85,2011
1,Sarasota County,Florida,227606,5.2,213400,378404,52.1,227606,128934,40322,103990,49212,62148,33096,65266,32221.0,1013,0.4830,372765,18202,12,115,2011
2,Citrus County,Florida,77587,5.2,134800,141157,53.8,77587,49936,9979,39352,38189,45921,22939,20563,23876.0,770,0.4390,138733,7956,12,17,2011
3,Santa Rosa County,Florida,64066,5.8,173400,150073,39.0,64066,42695,13261,41824,55913,63796,26507,14132,32870.0,956,0.4075,146588,7155,12,113,2011
4,Broward County,Florida,809226,4.7,225300,1742012,39.4,809226,453419,211618,417862,51782,62852,28720,247175,34312.0,1162,0.4720,1725510,99062,12,11,2011


In [119]:
finaldf.shape

(5151, 23)

In [120]:
finaldf.to_excel("../data/raw_data/AllData_By_County.xlsx")